In [1]:
import pdfplumber
from pdf2image import convert_from_path
import openai
import os
import base64
import json
import pandas as pd
from dotenv import load_dotenv
from io import BytesIO
import re

def try_fix_json(text):
    if text.count("{") > text.count("}"):
        text += "}" * (text.count("{") - text.count("}"))
    if text.count("[") > text.count("]"):
        text += "]" * (text.count("[") - text.count("]"))
    text = re.sub(r',([\s]*[}}\]])', r'\1', text)
    return text

def normalize_json_rows(data):
    if isinstance(data, list) and all(isinstance(row, dict) for row in data):
        all_keys = set()
        for row in data:
            all_keys.update(row.keys())
        all_keys = list(all_keys)
        normalized = []
        for row in data:
            normalized.append({k: row.get(k, "") for k in all_keys})
        return normalized
    return data

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=OPENAI_API_KEY)

# Palabras clave para cada tabla
tables_to_find = {
    "key_financial_metrics": "KEY FINANCIAL METRICS",
    "key_operational_metrics": "KEY OPERATIONAL METRICS",
    "robust_cash_flow": "ROBUST CASH FLOW GENERATION FROM ELG"  # <- la nueva tabla
}
PROMPT = """
¿Hay una tabla con el encabezado '{header}' en esta imagen?
Si existe, extrae TODOS los datos de la tabla, incluyendo los nombres de las métricas (encabezados de fila y columna), en formato JSON estructurado como una lista de diccionarios (uno por fila), donde cada diccionario tenga las claves: nombre de la métrica y sus valores. No inventes nada.
Si no existe, responde solo con 'NO TABLE.
"""

start_page = 20  # Python indexa desde 0

# --- NUEVO: Recorre todos los PDFs en la carpeta ---
DATA_DIR = "data"
for filename in os.listdir(DATA_DIR):
    if filename.lower().endswith(".pdf"):
        PDF_PATH = os.path.join(DATA_DIR, filename)
        print("\n--- Procesando:", filename, "---")
        # Extraer Q y año del nombre del PDF
        match = re.search(r'(?:^|[\\/_-])(?P<year>20\d{2})[^\dA-Za-z]*(?P<q>q[1-4])', filename, re.IGNORECASE)
        if not match:
            match = re.search(r'(?P<q>q[1-4])[^\dA-Za-z]*(?P<year>20\d{2})', filename, re.IGNORECASE)
        if match:
            trimestre = match.group('q').upper()
            anio = match.group('year')
        else:
            trimestre = "QX"
            anio = "YYYY"
        
        try:
            with pdfplumber.open(PDF_PATH) as pdf:
                pages = list(pdf.pages[start_page:])
                page_numbers = list(range(start_page + 1, start_page + 1 + len(pages)))  # 1-based

            for table_key, table_header in tables_to_find.items():
                table_found = False
                for page_idx, page_num in enumerate(page_numbers):
                    # Convierte la página a imagen EN MEMORIA
                    page_img = convert_from_path(PDF_PATH, first_page=page_num, last_page=page_num, dpi=150)[0]
                    buffer = BytesIO()
                    page_img.save(buffer, format="PNG")
                    img_base64 = base64.b64encode(buffer.getvalue()).decode("utf-8")
                    prompt = PROMPT.format(header=table_header)
                    response = client.chat.completions.create(
                        model="gpt-4o",
                        messages=[
                            {
                                "role": "user",
                                "content": [
                                    {"type": "text", "text": prompt},
                                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{img_base64}"}}
                                ]
                            }
                        ],
                        max_tokens=4096
                    )
                    content = response.choices[0].message.content.strip()
                    print(f"{table_key} - Página {page_num} respuesta GPT:\n{content[:300]}...\n")  # Solo muestra inicio

                    if "NO TABLE" not in content and "no table" not in content.lower():
                        if "```json" in content:
                            content = content.split("```json")[-1].split("```")[0]
                        elif "```" in content:
                            content = content.split("```")[-1].split("```")[0]
                        fixed_content = try_fix_json(content)
                        try:
                            data = json.loads(fixed_content)
                            data = normalize_json_rows(data)
                            df = pd.DataFrame(data)
                            # Guarda el Excel en una subcarpeta, nombrando por archivo, Q y año
                            out_name = f"{os.path.splitext(filename)[0]}_tabla_{table_key}_{trimestre}_{anio}.xlsx"
                            df.to_excel(os.path.join(DATA_DIR, out_name), index=False)
                            print(f"¡Tabla '{table_header}' encontrada en página {page_num} y guardada como {out_name}!")
                            print(df.head())
                            table_found = True
                            break
                        except Exception as e:
                            print(f"No se pudo parsear el JSON para la tabla '{table_key}'. Revisa la respuesta de OpenAI manualmente.")
                            print("Error:", e)
                            print("JSON crudo:\n", fixed_content[:1000])
                            break
                if not table_found:
                    print(f"No se encontró la tabla '{table_header}' en {filename}.")
        except Exception as err:
            print(f"Error procesando {filename}: {err}")



--- Procesando: 2021-q4-txg-cp.pdf ---
key_financial_metrics - Página 21 respuesta GPT:
NO TABLE...

key_financial_metrics - Página 22 respuesta GPT:
NO TABLE....

key_financial_metrics - Página 23 respuesta GPT:
```json
[
    {
        "Metric": "Gold sold (oz)",
        "Q1 2020": "108,064",
        "Q2 2020": "63,147",
        "Q3 2020": "133,036",
        "Q4 2020": "133,063",
        "Q1 2021": "129,019",
        "Q2 2021": "111,424",
        "Q3 2021": "118,989",
        "Q4 2021": "109,391",
        "...

¡Tabla 'KEY FINANCIAL METRICS' encontrada en página 23 y guardada como 2021-q4-txg-cp_tabla_key_financial_metrics_Q4_2021.xlsx!
   Q1 2020  Q1 2021  Q3 2021  Q4 2020  Q3 2020  FY 2020  \
0  108,064  129,019  118,989  133,063  133,036  437,310   
1   $1,571   $1,778   $1,786   $1,847   $1,884   $1,771   
2   $1,333   $1,022   $1,198   $1,075   $1,154   $1,217   
3     $794     $580     $727     $579     $633     $672   
4     $975     $854     $900     $886     $877     $924   